<a href="https://colab.research.google.com/github/JohnnyPeng123/NLP-USYD/blob/master/Lab06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab06

# POS Tagging
POS tagging is the process of marking up a word in a corpus to a corresponding part of a speech tag, based on its context and definition. This task is not straightforward, as a particular word may have a different part of speech based on the context in which the word is used

## Regular Expression Tagger

The regular expression tagger assigns tags to tokens on the basis of matching patterns. For instance, we might guess that any word ending in ed is the past participle of a verb, and any word ending with 's is a possessive noun. We can express these as a list of regular expressions: 



In [0]:
import nltk

# Downloading required corpus
nltk.download('punkt')
nltk.download('brown')

from nltk import word_tokenize
from nltk.corpus import brown

brown_tagged_sents = brown.tagged_sents(categories='news')
brown_sents = brown.sents(categories='news')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [0]:
 # Define regular expression patterns
 patterns = [
        (r'.*ing$', 'VBG'),               # gerunds
        (r'.*ed$', 'VBD'),                # simple past
        (r'.*es$', 'VBZ'),                # 3rd singular present
        (r'.*ould$', 'MD'),               # modals
        (r'.*\'s$', 'NN$'),               # possessive nouns
        (r'.*s$', 'NNS'),                 # plural nouns
        (r'^-?[0-9]+(.[0-9]+)?$', 'CD'),  # cardinal numbers
        (r'.*', 'NN')                     # nouns (default)
    ]

In [0]:
# Build regular expression tagger using the defined patterns
regexp_tagger = nltk.RegexpTagger(patterns)

# Print one of the sentences
print(brown_sents[3])
# Print one of the tagged sentences
print(regexp_tagger.tag(brown_sents[3]))

['``', 'Only', 'a', 'relative', 'handful', 'of', 'such', 'reports', 'was', 'received', "''", ',', 'the', 'jury', 'said', ',', '``', 'considering', 'the', 'widespread', 'interest', 'in', 'the', 'election', ',', 'the', 'number', 'of', 'voters', 'and', 'the', 'size', 'of', 'this', 'city', "''", '.']
[('``', 'NN'), ('Only', 'NN'), ('a', 'NN'), ('relative', 'NN'), ('handful', 'NN'), ('of', 'NN'), ('such', 'NN'), ('reports', 'NNS'), ('was', 'NNS'), ('received', 'VBD'), ("''", 'NN'), (',', 'NN'), ('the', 'NN'), ('jury', 'NN'), ('said', 'NN'), (',', 'NN'), ('``', 'NN'), ('considering', 'VBG'), ('the', 'NN'), ('widespread', 'NN'), ('interest', 'NN'), ('in', 'NN'), ('the', 'NN'), ('election', 'NN'), (',', 'NN'), ('the', 'NN'), ('number', 'NN'), ('of', 'NN'), ('voters', 'NNS'), ('and', 'NN'), ('the', 'NN'), ('size', 'NN'), ('of', 'NN'), ('this', 'NNS'), ('city', 'NN'), ("''", 'NN'), ('.', 'NN')]


In [0]:
# Evaluate the tagger (Calculate the accuracy/performance)
regexp_tagger.evaluate(brown_tagged_sents)

0.20326391789486245

In [0]:
raw = 'This race is awesome, I want to race too'
tokens = word_tokenize(raw)

print(regexp_tagger.tag(tokens))

[('This', 'NNS'), ('race', 'NN'), ('is', 'NNS'), ('awesome', 'NN'), (',', 'NN'), ('I', 'NN'), ('want', 'NN'), ('to', 'NN'), ('race', 'NN'), ('too', 'NN')]


# Hidden Markov Models 

A hidden Markov model (HMM) allows us to talk about both observed events Hidden
Markov model (like words that we see in the input) and hidden events (like part-of-speech tags) that we think of as causal factors in our probabilistic model.

In [0]:
# Hidden Markov Models in Python
# Katrin Erk, March 2013 updated March 2016
#
# This HMM addresses the problem of part-of-speech tagging. It estimates
# the probability of a tag sequence for a given word sequence as follows:
#
# Say words = w1....wN
# and tags = t1..tN
#
# then
# P(tags | words) is_proportional_to  product P(ti | t{i-1}) P(wi | ti)
#
# To find the best tag sequence for a given sequence of words,
# we want to find the tag sequence that has the maximum P(tags | words)
import nltk
import sys
nltk.download('brown')

from nltk.corpus import brown
from nltk.corpus import treebank


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [0]:
# Estimating P(wi | ti) from corpus data using Maximum Likelihood Estimation (MLE):
# P(wi | ti) = count(wi, ti) / count(ti)
#
# We add an artificial "start" tag at the beginning of each sentence, and
# We add an artificial "end" tag at the end of each sentence.
# So we start out with the brown tagged sentences,
# add the two artificial tags,
# and then make one long list of all the tag/word pairs.

brown_tags_words = []
brown_tagged_sents = brown.tagged_sents()

for sent in brown_tagged_sents:
    # sent is a list of word/tag pairs
    # add START/START at the beginning
    brown_tags_words.append( ("START", "START") )
    # then all the tag/word pairs for the word/tag pairs in the sentence.
    # shorten tags to 2 characters each
    brown_tags_words.extend([ (tag[:2], word) for (word, tag) in sent ])
    # then END/END
    brown_tags_words.append( ("END", "END") )

# conditional frequency distribution
cfd_tagwords = nltk.ConditionalFreqDist(brown_tags_words)
# conditional probability distribution
cpd_tagwords = nltk.ConditionalProbDist(cfd_tagwords, nltk.MLEProbDist)

print("The probability of an adjective (JJ) being 'new' is", cpd_tagwords["JJ"].prob("new"))
print("The probability of a verb (VB) being 'duck' is", cpd_tagwords["VB"].prob("duck"))

# Estimating P(ti | t{i-1}) from corpus data using Maximum Likelihood Estimation (MLE):
# P(ti | t{i-1}) = count(t{i-1}, ti) / count(t{i-1})
brown_tags = [tag for (tag, word) in brown_tags_words ]

# make conditional frequency distribution:
# count(t{i-1} ti)
cfd_tags= nltk.ConditionalFreqDist(nltk.bigrams(brown_tags))
# make conditional probability distribution, using
# maximum likelihood estimate:
# P(ti | t{i-1})
cpd_tags = nltk.ConditionalProbDist(cfd_tags, nltk.MLEProbDist)

print("If we have just seen 'DT', the probability of 'NN' is", cpd_tags["DT"].prob("NN"))
print( "If we have just seen 'VB', the probability of 'JJ' is", cpd_tags["VB"].prob("DT"))
print( "If we have just seen 'VB', the probability of 'NN' is", cpd_tags["VB"].prob("NN"))


The probability of an adjective (JJ) being 'new' is 0.01472344917632025
The probability of a verb (VB) being 'duck' is 6.042713350943527e-05
If we have just seen 'DT', the probability of 'NN' is 0.5057722522030194
If we have just seen 'VB', the probability of 'JJ' is 0.016885067592065053
If we have just seen 'VB', the probability of 'NN' is 0.10970977711020183


## Viterbi Algorithm

In [0]:
#####
# Viterbi:
# If we have a word sequence, what is the best tag sequence?
#
# The method above lets us determine the probability for a single tag sequence.
# But in order to find the best tag sequence, we need the probability
# for _all_ tag sequence.
# What Viterbi gives us is just a good way of computing all those many probabilities
# as fast as possible.

# what is the list of all tags?
distinct_tags = set(brown_tags)

sentence = ["This", "race", "is", "awesome", ",", "I", "want", "to", "race", "too" ]
#sentence = ["I", "saw", "her", "duck" ]
sentlen = len(sentence)

# viterbi:
# for each step i in 1 .. sentlen,
# store a dictionary
# that maps each tag X
# to the probability of the best tag sequence of length i that ends in X
viterbi = [ ]

# backpointer:
# for each step i in 1..sentlen,
# store a dictionary
# that maps each tag X
# to the previous tag in the best tag sequence of length i that ends in X
backpointer = [ ]

first_viterbi = { }
first_backpointer = { }
for tag in distinct_tags:
    # don't record anything for the START tag
    if tag == "START": continue
    first_viterbi[ tag ] = cpd_tags["START"].prob(tag) * cpd_tagwords[tag].prob( sentence[0] )
    first_backpointer[ tag ] = "START"

print(first_viterbi)
print(first_backpointer)
    
viterbi.append(first_viterbi)
backpointer.append(first_backpointer)

currbest = max(first_viterbi.keys(), key = lambda tag: first_viterbi[ tag ])
print( "Word", "'" + sentence[0] + "'", "current best two-tag sequence:", first_backpointer[ currbest], currbest)
# print( "Word", "'" + sentence[0] + "'", "current best tag:", currbest)

for wordindex in range(1, len(sentence)):
    this_viterbi = { }
    this_backpointer = { }
    prev_viterbi = viterbi[-1]
    
    for tag in distinct_tags:
        # don't record anything for the START tag
        if tag == "START": continue

        # if this tag is X and the current word is w, then 
        # find the previous tag Y such that
        # the best tag sequence that ends in X
        # actually ends in Y X
        # that is, the Y that maximizes
        # prev_viterbi[ Y ] * P(X | Y) * P( w | X)
        # The following command has the same notation
        # that you saw in the sorted() command.
        best_previous = max(prev_viterbi.keys(),
                            key = lambda prevtag: \
            prev_viterbi[ prevtag ] * cpd_tags[prevtag].prob(tag) * cpd_tagwords[tag].prob(sentence[wordindex]))

        # Instead, we can also use the following longer code:
        # best_previous = None
        # best_prob = 0.0
        # for prevtag in distinct_tags:
        #    prob = prev_viterbi[ prevtag ] * cpd_tags[prevtag].prob(tag) * cpd_tagwords[tag].prob(sentence[wordindex])
        #    if prob > best_prob:
        #        best_previous= prevtag
        #        best_prob = prob
        #
        this_viterbi[ tag ] = prev_viterbi[ best_previous] * \
            cpd_tags[ best_previous ].prob(tag) * cpd_tagwords[ tag].prob(sentence[wordindex])
        this_backpointer[ tag ] = best_previous

    currbest = max(this_viterbi.keys(), key = lambda tag: this_viterbi[ tag ])
    print( "Word", "'" + sentence[ wordindex] + "'", "current best two-tag sequence:", this_backpointer[ currbest], currbest)
    # print( "Word", "'" + sentence[ wordindex] + "'", "current best tag:", currbest)


    # done with all tags in this iteration
    # so store the current viterbi step
    viterbi.append(this_viterbi)
    backpointer.append(this_backpointer)


# done with all words in the sentence.
# now find the probability of each tag
# to have "END" as the next tag,
# and use that to find the overall best sequence
prev_viterbi = viterbi[-1]
best_previous = max(prev_viterbi.keys(),
                    key = lambda prevtag: prev_viterbi[ prevtag ] * cpd_tags[prevtag].prob("END"))

prob_tagsequence = prev_viterbi[ best_previous ] * cpd_tags[ best_previous].prob("END")

# best tagsequence: we store this in reverse for now, will invert later
best_tagsequence = [ "END", best_previous ]
# invert the list of backpointers
backpointer.reverse()

# go backwards through the list of backpointers
# (or in this case forward, because we have inverter the backpointer list)
# in each case:
# the following best tag is the one listed under
# the backpointer for the current best tag
current_best_tag = best_previous
for bp in backpointer:
    best_tagsequence.append(bp[current_best_tag])
    current_best_tag = bp[current_best_tag]

best_tagsequence.reverse()
print( "The sentence was:", end = " ")
for w in sentence: print( w, end = " ")
print("\n")
print( "The best tag sequence is:", end = " ")
for t in best_tagsequence: print (t, end = " ")
print("\n")
print( "The probability of the best tag sequence is:", prob_tagsequence)



{'TO': 0.0, 'CS': 0.0, 'MD': 0.0, 'RN': 0.0, "'": 0.0, ',-': 0.0, 'WR': 0.0, ',': 0.0, 'NR': 0.0, 'BE': 0.0, 'AT': 0.0, 'CC': 0.0, 'FW': 0.0, 'PP': 0.0, 'NP': 0.0, 'END': 0.0, 'WP': 0.0, 'WD': 0.0, '.-': 0.0, ':': 0.0, '.': 0.0, 'VB': 0.0, '(-': 0.0, ':-': 0.0, '``': 0.0, 'DO': 0.0, 'HV': 0.0, 'DT': 0.0033218181276236437, ')-': 0.0, 'AB': 0.0, 'IN': 0.0, '--': 0.0, 'WQ': 0.0, '*-': 0.0, '(': 0.0, 'OD': 0.0, 'PN': 0.0, 'UH': 0.0, 'CD': 0.0, 'RB': 0.0, '*': 0.0, 'EX': 0.0, "''": 0.0, 'AP': 0.0, ')': 0.0, 'QL': 0.0, 'NI': 0.0, 'RP': 0.0, 'JJ': 0.0, 'NN': 0.0}
{'TO': 'START', 'CS': 'START', 'MD': 'START', 'RN': 'START', "'": 'START', ',-': 'START', 'WR': 'START', ',': 'START', 'NR': 'START', 'BE': 'START', 'AT': 'START', 'CC': 'START', 'FW': 'START', 'PP': 'START', 'NP': 'START', 'END': 'START', 'WP': 'START', 'WD': 'START', '.-': 'START', ':': 'START', '.': 'START', 'VB': 'START', '(-': 'START', ':-': 'START', '``': 'START', 'DO': 'START', 'HV': 'START', 'DT': 'START', ')-': 'START', 'AB'

The code is implemented by [Katrin Erk](http://www.katrinerk.com/courses/python-worksheets/hidden-markov-models-for-pos-tagging-in-python)

##  Train HMM Tagger with NLTK HMM Trainer

In [0]:
# Pretagged training data
brown_tagged_sents = brown.tagged_sents()

print(brown_tagged_sents)

[[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN'), ("Atlanta's", 'NP$'), ('recent', 'JJ'), ('primary', 'NN'), ('election', 'NN'), ('produced', 'VBD'), ('``', '``'), ('no', 'AT'), ('evidence', 'NN'), ("''", "''"), ('that', 'CS'), ('any', 'DTI'), ('irregularities', 'NNS'), ('took', 'VBD'), ('place', 'NN'), ('.', '.')], [('The', 'AT'), ('jury', 'NN'), ('further', 'RBR'), ('said', 'VBD'), ('in', 'IN'), ('term-end', 'NN'), ('presentments', 'NNS'), ('that', 'CS'), ('the', 'AT'), ('City', 'NN-TL'), ('Executive', 'JJ-TL'), ('Committee', 'NN-TL'), (',', ','), ('which', 'WDT'), ('had', 'HVD'), ('over-all', 'JJ'), ('charge', 'NN'), ('of', 'IN'), ('the', 'AT'), ('election', 'NN'), (',', ','), ('``', '``'), ('deserves', 'VBZ'), ('the', 'AT'), ('praise', 'NN'), ('and', 'CC'), ('thanks', 'NNS'), ('of', 'IN'), ('the', 'AT'), ('City', 'NN-TL'), ('of', 'IN-TL'), ('Atlant

In [0]:
# Import HMM module
from nltk.tag import hmm

# Setup a trainer with default(None) values
# And train with the data
trainer = hmm.HiddenMarkovModelTrainer()
trained_tagger = trainer.train_supervised(brown_tagged_sents)

print (trained_tagger)
# Prints the basic data about the tagger

tokens = word_tokenize("This race is awesome, I want to race too")
print(trained_tagger.tag(tokens))



<HiddenMarkovModelTagger 472 states and 56057 output symbols>
[('This', 'DT'), ('race', 'NN'), ('is', 'BEZ'), ('awesome', 'JJ'), (',', ','), ('I', 'PPSS'), ('want', 'VB'), ('to', 'TO'), ('race', 'VB'), ('too', 'QL')]


# Bi-LSTM based POS Tagger (Pytorch)

In this example, we would try to train PoS tagger using Bi-LSTM. 

![alt text](https://usydnlpgroup.files.wordpress.com/2020/03/bi-lstm_nton-e1586049916759.png)

## Training data

In [0]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize

nltk.download('treebank')
from nltk.corpus import treebank

import numpy as np
from sklearn.model_selection import train_test_split
 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


In [0]:
# Retrieve tagged sentences from treebank corpus
tagged_sentences = nltk.corpus.treebank.tagged_sents()
 
print(tagged_sentences[0])
print("Tagged sentences: ", len(tagged_sentences))
print("Tagged words:", len(nltk.corpus.treebank.tagged_words()))
#tagged_words(): list of (str,str) tuple

[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]
Tagged sentences:  3914
Tagged words: 100676


In [0]:
sentences, sentence_tags =[], [] 
for tagged_sentence in tagged_sentences:
    #The zip() function returns a zip object, which is an iterator of tuples where the first item in each passed iterator is paired together, 
    #and then the second item in each passed iterator are paired together etc.
    sentence, tags = zip(*tagged_sentence)
    sentences.append(np.array(sentence))
    sentence_tags.append(np.array(tags))
 
print(sentences[5])
print(sentence_tags[5])

['Lorillard' 'Inc.' ',' 'the' 'unit' 'of' 'New' 'York-based' 'Loews'
 'Corp.' 'that' '*T*-2' 'makes' 'Kent' 'cigarettes' ',' 'stopped' 'using'
 'crocidolite' 'in' 'its' 'Micronite' 'cigarette' 'filters' 'in' '1956'
 '.']
['NNP' 'NNP' ',' 'DT' 'NN' 'IN' 'JJ' 'JJ' 'NNP' 'NNP' 'WDT' '-NONE-' 'VBZ'
 'NNP' 'NNS' ',' 'VBD' 'VBG' 'NN' 'IN' 'PRP$' 'NN' 'NN' 'NNS' 'IN' 'CD'
 '.']


In [0]:
(train_sentences, 
 test_sentences, 
 train_tags, 
 test_tags) = train_test_split(sentences, sentence_tags, test_size=0.2)

### Making vocabs with special tokens

*PAD: Padding*
*OOV: Out Of Vocabulary*

In [0]:
words, tags = set([]), set([])
 
for s in train_sentences:
    for w in s:
        words.add(w.lower())

for ts in train_tags:
    for t in ts:
        tags.add(t)

word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs
 
tag2index = {t: i + 2 for i, t in enumerate(list(tags))}
tag2index['-PAD-'] = 0  # The special value used to padding
tag2index['-OOV-'] = 1  # The special value used for OOVs

def tag_to_index(tag):
    if tag in tag2index:
        return tag2index[tag]
    else:
        return tag2index['-OOV-']

In [0]:
train_sentences_X, test_sentences_X, train_tags_y, test_tags_y = [], [], [], []

for s in train_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])

    train_sentences_X.append(s_int)

for s in test_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])

    test_sentences_X.append(s_int)

for s in train_tags:
    train_tags_y.append([tag_to_index(t) for t in s])

for s in test_tags:
    test_tags_y.append([tag_to_index(t) for t in s])

print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

[1055, 110, 8619, 9873, 9593, 830, 8549, 9780, 7221, 6976, 1255, 2877, 3572, 9695, 2353, 2642, 1506, 760, 9048, 4326, 1190, 3566, 8262, 2444, 4094]
[1208, 1991, 5792, 7154, 9780, 1096, 1236, 8223, 2069, 1340, 9780, 7395, 5790, 982, 2856, 3750, 3572, 4035, 2327, 9780, 5900, 331, 5790, 4108, 5735, 5039, 1, 7395, 2869, 1340, 6095, 5790, 5045, 7654, 5913, 2415, 509, 5913, 4925, 3630, 7115, 441, 7154, 6042, 4094]
[47, 47, 14, 9, 28, 42, 27, 26, 21, 47, 12, 9, 27, 13, 41, 31, 23, 42, 21, 21, 3, 31, 23, 47, 22]
[27, 26, 47, 27, 26, 45, 47, 47, 47, 27, 26, 21, 8, 47, 14, 9, 27, 45, 21, 26, 45, 47, 8, 46, 42, 31, 41, 21, 14, 27, 47, 8, 9, 31, 2, 21, 42, 2, 23, 30, 45, 14, 27, 47, 22]


### Getting max length of sequence

In [0]:
MAX_LENGTH = len(max(train_sentences_X, key=len))
print(MAX_LENGTH) 

271


### Add PAD by using torch pad_sequence
Due to the limitation of Pytorch pad_sequence, we can't assign the max_length, the max_length is calculated by pad_sequence itself. You can try to add padding manually by using for loop.

In [0]:
import torch
new_train_sentences_X = [torch.from_numpy(np.array(l)) for l in train_sentences_X]
new_test_sentences_X = [torch.from_numpy(np.array(l)) for l in test_sentences_X]
new_train_tags_y = [torch.from_numpy(np.array(l)) for l in train_tags_y]
new_test_tags_y = [torch.from_numpy(np.array(l)) for l in test_tags_y]

In [0]:
from torch.nn.utils.rnn import pad_sequence
after_pad = pad_sequence(new_train_sentences_X+new_test_sentences_X+new_train_tags_y+new_test_tags_y,batch_first=True)
train_sentences_X_pad = after_pad[:len(new_train_sentences_X)]
test_sentences_X_pad = after_pad[len(new_train_sentences_X):len(new_train_sentences_X)+len(new_test_sentences_X)]
train_tags_y_pad = after_pad[len(new_train_sentences_X)+len(new_test_sentences_X):-len(new_test_tags_y)]
test_tags_y_pad = after_pad[-len(new_test_tags_y):]

### Build Dataset and Dataloader for training data

In [0]:
#More detailed info about the TensorDataset, https://pytorch.org/docs/1.1.0/_modules/torch/utils/data/dataset.html#TensorDataset
from torch.utils.data import TensorDataset
train_data = TensorDataset(train_sentences_X_pad, train_tags_y_pad)

batch_size = 128
#More detailed info about the dataLoader, https://pytorch.org/docs/1.1.0/_modules/torch/utils/data/dataloader.html
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=False)

## Pytorch Model (Bidirectional LSTM)

In [0]:
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,batch_first=True, bidirectional=True)  
        self.hidden2tag = nn.Linear(hidden_dim * 2, tagset_size)


    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds)
        tag_space = self.hidden2tag(lstm_out) 
        tag_scores = F.log_softmax(tag_space, dim=-1)     
        return tag_scores


EMBEDDING_DIM = 128
HIDDEN_DIM = 256

model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word2index), len(tag2index)).cuda()
loss_function = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


for epoch in range(40):  
    loss_now = 0.0
    acc = 0

    for sentence,targets in train_loader:
        sentence = sentence.cuda()
        targets = targets.cuda()

        model.zero_grad()
        model.train()       
        tag_scores = model(sentence)

        # loss = loss_function(tag_scores, targets)
        loss = loss_function(tag_scores.view(-1,tag_scores.size(-1)), targets.view(-1))
        loss.backward()
        optimizer.step()

        loss_now+=loss.item()

        model.eval()
        tag_scores = model(sentence)
        _, predicted = torch.max(tag_scores, -1)
        prediction = predicted.view(-1).cpu().numpy()
        t = targets.view(-1).cpu().numpy()
        # Note: The training accuracy here is calculated with "PAD", which means most of pos tag will be "0".
        acc = acc+accuracy_score(prediction,t)*len(prediction) 
    print('Epoch: %d, training loss: %.4f, training acc: %.2f%%'%(epoch+1,loss_now,100*acc/len(train_sentences_X)/MAX_LENGTH))



Epoch: 1, training loss: 16.8474, training acc: 91.00%
Epoch: 2, training loss: 7.4900, training acc: 92.74%
Epoch: 3, training loss: 6.0220, training acc: 94.20%
Epoch: 4, training loss: 5.1115, training acc: 94.89%
Epoch: 5, training loss: 4.3215, training acc: 95.43%
Epoch: 6, training loss: 3.6197, training acc: 96.22%
Epoch: 7, training loss: 3.0292, training acc: 96.82%
Epoch: 8, training loss: 2.5696, training acc: 97.29%
Epoch: 9, training loss: 2.1989, training acc: 97.68%
Epoch: 10, training loss: 1.8934, training acc: 97.99%
Epoch: 11, training loss: 1.6402, training acc: 98.27%
Epoch: 12, training loss: 1.4292, training acc: 98.49%
Epoch: 13, training loss: 1.2518, training acc: 98.70%
Epoch: 14, training loss: 1.1010, training acc: 98.87%
Epoch: 15, training loss: 0.9717, training acc: 99.00%
Epoch: 16, training loss: 0.8608, training acc: 99.12%
Epoch: 17, training loss: 0.7655, training acc: 99.22%
Epoch: 18, training loss: 0.6831, training acc: 99.31%
Epoch: 19, trainin

## Test with the test set

In [0]:
model.eval()
sentence = test_sentences_X_pad.cuda()
tag_scores = model(sentence)
_, predicted = torch.max(tag_scores, -1)
predicted = predicted.cpu().numpy()

# cut off the PAD part
test_len_list = [len(s) for s in test_sentences_X]
actual_predicted_list= []
for i in range(predicted.shape[0]):
    actual_predicted_list+=list(predicted[i])[:test_len_list[i]]

# get actual tag list
actual_tags = sum(test_tags_y, [])

print('Test Accuracy: %.2f%%'%(accuracy_score(actual_predicted_list,actual_tags)*100))

Test Accuracy: 91.42%


# Exercise
In this exercise, you are to classify part-of-speech(pos) tags on user-defined sentences using the Bi-LSTM model trained right before the exercise. You should complete the below "Prediction Result to PoS Tags" section


## Testing with the sentence

### Preprocess and Predcition

In [0]:
test_samples = [
    word_tokenize("This race is awesome, I want to race too."),
    word_tokenize("That race is silly, I do not want to race.")
]

# Converting sentence (tokens) word to index
test_samples_X = []
for s in test_samples:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    test_samples_X.append(s_int)

# manually add PAD
test_samples_X_pad = []
for l in test_samples_X:
    test_samples_X_pad.append(l+[0]*(MAX_LENGTH-len(l)))

index2tag = {i: t for t, i in tag2index.items()}

model.eval()
sentence = torch.from_numpy(np.array(test_samples_X_pad)).cuda()
predictions = model(sentence)
_, predictions = torch.max(predictions, 1)
predictions = predictions.cpu().numpy()

### Prediction Result to PoS Tags [Complete this part]

In [0]:
#decode the result to have actual tags
def decode_result(predictions, test_samples_X, index2tag):
    token_sequences = []
    ## write your codes here


    return token_sequences

print(test_samples)
print(decode_result(predictions, test_samples_X, index2tag ))